<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_MYVERSION_OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/tools/OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool Tutorial

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [20]:
!pip install llama-index-readers-wikipedia -q

In [22]:
!pip install llama-index -q

In [11]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

### Define Tool

We first define the `WikipediaReader`. Note that the `load_data` interface to `WikipediaReader` takes in a list of `pages`. By default, this queries the Wikipedia search endpoint which will autosuggest the relevant pages.

We then wrap it into our `OnDemandLoaderTool`.

By default since we don't specify the `index_cls`, a simple vector store index is initialized.

In [24]:
!pip install wikipedia -q
!pip install langchain -q

In [13]:
reader = WikipediaReader()

In [14]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

#### Testing

We can try running the tool by itself (or as a LangChain tool), just to showcase what the interface is like!

Note that besides the arguments required for the data loader, the tool also takes in a `query_str` which will be
the query against the index.

In [28]:
#added by Frank Morales(FM) 22/02/2024
!pip install openai  --root-user-action=ignore -q
!pip install colab-env --upgrade --quiet --root-user-action=ignore -q

# OpenAI Settings

In [5]:
#added by Frank Morales(FM) 22/02/2024
import warnings
warnings.filterwarnings('ignore')

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# run tool by itself
#added by Frank Morales(FM) 22/02/2024
tool(["Montreal"], query_str="What is the best restaurant in Montreal?")

In [15]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [16]:
lc_tool.run(
    tool_input={
        "pages": ["Montreal"],
        "query_str": "What is the best restaurant in Montreal?",
    }
)

I do not have information regarding the best restaurant in Montreal based on the context provided.

ToolOutput(content='I do not have information regarding the best restaurant in Montreal based on the context provided.', tool_name='Wikipedia Tool', raw_input={'query': 'What is the best restaurant in Montreal?'}, raw_output=Response(response='I do not have information regarding the best restaurant in Montreal based on the context provided.', source_nodes=[NodeWithScore(node=TextNode(id_='7a291fe4-3e93-4781-99b7-f03f132a4e5e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7954681', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='2c83d3017e2b5e6fc169d91f22b0db0b2051c8680ec8b8b3c6165ff2d9afbd63'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='1cd592a1-0cd6-4196-baad-87cbfc833167', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='8faf4c36397eac44a173c0e165427e3a1685d179af1e84d8e9ed5e8a3f13d2db'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node

### Initialize LangChain Agent

For tutorial purposes, the agent just has access to one tool - the Wikipedia Reader

Note that we need to use Structured Tools from LangChain.

In [17]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

In [28]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)

llmGPT4 = ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)

In [29]:
agent = initialize_agent(
    [lc_tool],
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=False,
)

In [30]:
agentGPT4 = initialize_agent(
    [lc_tool],
    llm=llmGPT4,
    agent="structured-chat-zero-shot-react-description",
    verbose=False,
)

# Now let's run some queries!

The OnDemandLoaderTool allows the agent to simultaneously 1) load the data from Wikipedia, 2) query that data.

In [ ]:
agent.run("What is the best restaurant in Montreal?")

In [33]:
agentGPT4.run("What is the best restaurant in Montreal?")


Observation: Québécois restaurants such as Lafleur, Valentine, La Belle Province, and St-Hubert have played a significant role in showcasing traditional Québécois dishes and contributing to the culinary landscape of Québec. These restaurants have been instrumental in popularizing iconic dishes like poutine, tourtière, and guédille, offering locals and visitors alike a taste of authentic Québécois cuisine. Additionally, the influence of immigrants from various countries, particularly Italy, Greece, France, and Portugal, has led to the diversification of menus in Québécois restaurants, incorporating dishes like souvlaki pita, shawarmas, and other international flavors.
Thought:

"The 'best' restaurant can be subjective and depends on personal preferences, such as type of cuisine, ambiance, price range, etc. However, some of the most popular restaurants in Montreal for traditional Québécois cuisine include Lafleur, Valentine, La Belle Province, and St-Hubert. These restaurants have been instrumental in popularizing iconic dishes like poutine, tourtière, and guédille, offering locals and visitors alike a taste of authentic Québécois cuisine."